In [2]:
from PIL import Image
import numpy as np

def anahtar_uret(resim, blok_boyutu=4):
    print("Resimden anahtar üretiliyor...")
    anahtar_bitleri = []
    genislik, yukseklik = resim.size
    pikseller = np.array(resim)

    for y in range(0, yukseklik, blok_boyutu):
        for x in range(0, genislik, blok_boyutu):
            for dy in range(blok_boyutu):
                for dx in range(blok_boyutu):
                    if y + dy < yukseklik and x + dx < genislik:
                        anahtar_bitleri.append((pikseller[y + dy, x + dx, 0] >> 1) & 1)  # Kırmızı kanaldan ikinci LSB çıkar

    anahtar_baytlar = np.packbits(anahtar_bitleri)[:16]  # 128 bit (16 bayt) anahtar sağla
    print(f"Anahtar üretildi: {anahtar_baytlar}")
    return bytes(anahtar_baytlar)

def mesaj_gomme(resim, mesaj):
    print("Mesaj resme gömülüyor...")
    pikseller = np.array(resim, dtype=np.uint8)  # Doğru veri tipini sağla
    genislik, yukseklik = resim.size
    mesaj_bitleri = np.unpackbits(np.frombuffer(mesaj.encode(), dtype=np.uint8))
    print(f"Mesaj bitlere dönüştürüldü: {mesaj_bitleri}")

    # Adaptif gömme için rastgele bir sıra oluştur
    np.random.seed(42)  # Tekrar üretilebilirlik için sabit bir tohum kullan
    rastgele_pozisyonlar = np.random.permutation(genislik * yukseklik)

    bit_indeksi = 0
    for pos in rastgele_pozisyonlar:
        if bit_indeksi >= len(mesaj_bitleri):
            break
        y = pos // genislik
        x = pos % genislik
        # Mesaj bitini en düşük anlamlı bit (LSB) içine değil, ikinci LSB içine göm
        orijinal_piksel_degeri = int(pikseller[y, x, 2])  # Güvenlik için tamsayıya dönüştür
        pikseller[y, x, 2] = (orijinal_piksel_degeri & ~2) | (mesaj_bitleri[bit_indeksi] << 1)
        bit_indeksi += 1

    print(f"Mesaj gömme tamamlandı. Toplam gömülen bit sayısı: {bit_indeksi}")
    return Image.fromarray(pikseller)

def mesaj_cikarma(resim, mesaj_uzunlugu):
    print("Mesaj resimden çıkarılıyor...")
    pikseller = np.array(resim)
    genislik, yukseklik = resim.size

    # Çıkarma için aynı rastgele sırayı oluştur
    np.random.seed(42)  # Gömme işlemindeki aynı tohum kullanılır
    rastgele_pozisyonlar = np.random.permutation(genislik * yukseklik)

    mesaj_bitleri = []
    for pos in rastgele_pozisyonlar:
        if len(mesaj_bitleri) >= mesaj_uzunlugu * 8:
            break
        y = pos // genislik
        x = pos % genislik
        # İkinci en düşük anlamlı bitten bit çıkar
        mesaj_bitleri.append((pikseller[y, x, 2] & 2) >> 1)

    mesaj_baytlar = np.packbits(mesaj_bitleri)
    print(f"Çıkarılan bitler: {mesaj_bitleri[:mesaj_uzunlugu * 8]}...")
    return mesaj_baytlar.tobytes().decode()

if __name__ == "__main__":
    # Resim yükle
    girdi_resim_yolu = "lsb.png"
    print(f"{girdi_resim_yolu} yolundan resim yükleniyor...")
    resim = Image.open(girdi_resim_yolu)
    print("Resim başarıyla yüklendi.")

    # Adım 1: Resimden anahtar üret
    anahtar = anahtar_uret(resim)
    print("Üretilen Anahtar (referans için):", anahtar)

    # Adım 2: Mesajı resme göm
    mesaj = "Bu bir gizli mesajdır"
    print(f"Gömülecek mesaj: \"{mesaj}\"")
    stego_resim = mesaj_gomme(resim, mesaj)
    stego_resim_yolu = "stego_image.png"
    stego_resim.save(stego_resim_yolu)
    print(f"Stego resim {stego_resim_yolu} yoluna kaydedildi.")

    # Adım 3: Resimden mesaj çıkar
    cikarilan_mesaj = mesaj_cikarma(stego_resim, len(mesaj))
    print("Çıkarılan mesaj:", cikarilan_mesaj)

lsb.png yolundan resim yükleniyor...
Resim başarıyla yüklendi.
Resimden anahtar üretiliyor...
Anahtar üretildi: [225 148 210 140  47 252  47  86 219 177   8  35 253  20 139 162]
Üretilen Anahtar (referans için): b'\xe1\x94\xd2\x8c/\xfc/V\xdb\xb1\x08#\xfd\x14\x8b\xa2'
Gömülecek mesaj: "Bu bir gizli mesajdır"
Mesaj resme gömülüyor...
Mesaj bitlere dönüştürüldü: [0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 1
 0 0 1 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1
 1 1 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0
 1 0 1 1 0 0 1 0 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0]
Mesaj gömme tamamlandı. Toplam gömülen bit sayısı: 176
Stego resim stego_image.png yoluna kaydedildi.
Mesaj resimden çıkarılıyor...
Çıkarılan bitler: [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 